In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

opts_str = " ".join(
  [o for c, o in [(USE_COMPLEX, "--complex"), (USE_CLEAN, "--clean")] if c]
)

get_ipython().run_line_magic("run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}")

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx -np 4

"""
Stokes flow problem: Lid-driven cavity flow
Flow inside a square cavity with the top boundary moving at constant velocity
"""

from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

from dolfinx import fem, mesh, plot
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
import ufl
from ufl import div, dx, grad, inner

# Create mesh
msh = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

# Define function spaces
V = fem.functionspace(msh, ("Lagrange", 2, (msh.geometry.dim,)))  # Velocity (P2 vector)
Q = fem.functionspace(msh, ("Lagrange", 1))  # Pressure (P1 scalar)

# Define boundary conditions
def on_boundary_walls(x):
    """Bottom, left, right walls (no-slip)"""
    return np.logical_or.reduce([
        np.isclose(x[0], 0),  # Left
        np.isclose(x[0], 1),  # Right
        np.isclose(x[1], 0)   # Bottom
    ])

def on_lid(x):
    """Top lid (moving lid)"""
    return np.isclose(x[1], 1)

# Wall boundaries: velocity = 0 (no-slip)
u_walls = fem.Function(V)
u_walls.x.array[:] = 0.0

# Top lid: velocity = (1, 0) (moving right)
u_lid = fem.Function(V)

def lid_velocity(x):
    """Velocity profile of the top lid"""
    values = np.zeros((2, x.shape[1]))
    values[0] = 1.0  # x-direction velocity = 1
    values[1] = 0.0  # y-direction velocity = 0
    return values

u_lid.interpolate(lid_velocity)

# Create boundary condition objects
facets_walls = mesh.locate_entities_boundary(msh, msh.topology.dim - 1, on_boundary_walls)
facets_lid = mesh.locate_entities_boundary(msh, msh.topology.dim - 1, on_lid)

dofs_walls = fem.locate_dofs_topological(V, msh.topology.dim - 1, facets_walls)
dofs_lid = fem.locate_dofs_topological(V, msh.topology.dim - 1, facets_lid)

bc_walls = fem.dirichletbc(u_walls, dofs_walls)
bc_lid = fem.dirichletbc(u_lid, dofs_lid)

bcs = [bc_walls, bc_lid]

# Fix pressure (pressure = 0 at one point)
# Pressure is determined only up to a constant, so one point must be fixed
zero_pressure = fem.Function(Q)
zero_pressure.x.array[:] = 0.0

# Fix pressure at corner point
def corner(x):
    return np.logical_and(np.isclose(x[0], 0), np.isclose(x[1], 0))

dofs_pressure = fem.locate_dofs_geometrical(Q, corner)
bc_pressure = fem.dirichletbc(zero_pressure, dofs_pressure)

# Trial and test functions
u = ufl.TrialFunction(V)
p = ufl.TrialFunction(Q)
v = ufl.TestFunction(V)
q = ufl.TestFunction(Q)

# Bilinear form
a00 = inner(grad(u), grad(v)) * dx
a01 = -inner(p, div(v)) * dx
a10 = -inner(div(u), q) * dx
a11 = None  # No pressure-pressure coupling term

# Linear form (right-hand side - no external force in this case)
f = fem.Constant(msh, PETSc.ScalarType((0.0, 0.0)))  # External force = 0 (2D vector)
L0 = inner(f, v) * dx
L1 = inner(fem.Constant(msh, PETSc.ScalarType(0.0)), q) * dx

# Compile forms
a00 = fem.form(a00)
a01 = fem.form(a01)
a10 = fem.form(a10)
L0 = fem.form(L0)
L1 = fem.form(L1)

# Assemble block matrix
A00 = assemble_matrix(a00, bcs=[bc_walls, bc_lid])
A00.assemble()

A01 = assemble_matrix(a01)
A01.assemble()

A10 = assemble_matrix(a10)
A10.assemble()

# Assemble right-hand side vector
b0 = assemble_vector(L0)
apply_lifting(b0, [a00], [bcs])
b0.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(b0, bcs)

b1 = assemble_vector(L1)
apply_lifting(b1, [a10], [[]])
b1.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(b1, [bc_pressure])

# Construct block system
A = PETSc.Mat().createNest([[A00, A01], [A10, None]])
A.assemble()

b = PETSc.Vec().createNest([b0, b1])

# Solution vector
uh = fem.Function(V)
ph = fem.Function(Q)
x = PETSc.Vec().createNest([uh.x.petsc_vec, ph.x.petsc_vec])

# Solve linear system
ksp = PETSc.KSP().create(msh.comm)
ksp.setOperators(A)
ksp.setType("minres")
pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("mumps")

# Solve system
ksp.solve(b, x)

# Output results
if MPI.COMM_WORLD.rank == 0:
    print(f"Solver converged: {ksp.getConvergedReason()}")
    print(f"Number of iterations: {ksp.getIterationNumber()}")
    print(f"Residual norm: {ksp.getResidualNorm():.6e}")
    print("\nSolution computed successfully!")
    print("uh.x.array: velocity field")
    print("ph.x.array: pressure field")

# Save results (optional)
try:
    from dolfinx.io import XDMFFile

    # Create P1 function spaces for output (same degree as mesh)
    V_out = fem.functionspace(msh, ("Lagrange", 1, (msh.geometry.dim,)))
    Q_out = fem.functionspace(msh, ("Lagrange", 1))

    # Interpolate to P1 for visualization
    uh_out = fem.Function(V_out)
    uh_out.interpolate(uh)

    ph_out = fem.Function(Q_out)
    ph_out.interpolate(ph)

    # Write to file
    with XDMFFile(msh.comm, "velocity.xdmf", "w") as xdmf:
        xdmf.write_mesh(msh)
        xdmf.write_function(uh_out)
    with XDMFFile(msh.comm, "pressure.xdmf", "w") as xdmf:
        xdmf.write_mesh(msh)
        xdmf.write_function(ph_out)
    if MPI.COMM_WORLD.rank == 0:
        print("\nResults saved to velocity.xdmf and pressure.xdmf")
except ImportError:
    if MPI.COMM_WORLD.rank == 0:
        print("\nh5py is required to save XDMF files.")

Solver converged: 2
Number of iterations: 1
Residual norm: 1.035689e-14

Solution computed successfully!
uh.x.array: velocity field
ph.x.array: pressure field

Results saved to velocity.xdmf and pressure.xdmf


In [3]:
from google.colab import files

files.download('velocity.xdmf')
files.download('velocity.h5')
files.download('pressure.xdmf')
files.download('pressure.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>